## 1. Configuração Inicial

In [1]:
import warnings
import json
import numpy as np
from dotenv import load_dotenv
from openai import OpenAI

# Importando funções do módulo local
from embedding_utils import (
    get_embedding,
    cosine_similarity,
)

warnings.filterwarnings("ignore")

# Carrega variáveis do arquivo .env
load_dotenv()

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


In [2]:
# Cliente OpenAI
client = OpenAI()

print("✅ Cliente OpenAI configurado!")

✅ Cliente OpenAI configurado!


## 2. Carregando Base de Conhecimento

Vamos carregar os embeddings e chunks que geramos no notebook anterior.

In [3]:
# Carregando embeddings e metadados salvos
embeddings_chunks = np.load("embeddings_chunks.npy")
with open("metadados_chunks.json", "r", encoding="utf-8") as f:
    metadados = json.load(f)

# Carregando os chunks de texto
with open("chunks.json", "r", encoding="utf-8") as f:
    todos_chunks = json.load(f)

print(f"✅ Base de conhecimento carregada!")
print(f"   📚 {len(todos_chunks)} chunks disponíveis")
print(f"   📐 Dimensão dos embeddings: {embeddings_chunks.shape[1]}")

✅ Base de conhecimento carregada!
   📚 337 chunks disponíveis
   📐 Dimensão dos embeddings: 3072


## 3. Sistema RAG Básico

Vamos criar as funções principais do RAG.

In [4]:
def buscar_contexto(pergunta: str, top_k: int = 3) -> list:
    """
    Busca os chunks mais relevantes para uma pergunta.

    Args:
        pergunta: A pergunta em linguagem natural
        top_k: Número de chunks a retornar

    Returns:
        Lista de dicionários com chunk, fonte e similaridade
    """
    # Gera embedding da pergunta
    emb_pergunta = get_embedding(client, pergunta)

    # Calcula similaridade com todos os chunks
    resultados = []
    for i, emb_chunk in enumerate(embeddings_chunks):
        sim = cosine_similarity(emb_pergunta, emb_chunk)
        resultados.append(
            {
                "chunk": todos_chunks[i],
                "fonte": metadados[i]["pagina"],
                "url": metadados[i]["url"],
                "similaridade": sim,
            }
        )

    # Ordena por similaridade (maior primeiro)
    resultados.sort(key=lambda x: x["similaridade"], reverse=True)

    return resultados[:top_k]


print("✅ Função de busca definida!")

✅ Função de busca definida!


In [5]:
def gerar_resposta_rag(
    pergunta: str, top_k: int = 3, modelo: str = "gpt-4o-mini"
) -> dict:
    """
    Gera uma resposta usando RAG (Retrieval Augmented Generation).

    Args:
        pergunta: A pergunta do usuário
        top_k: Número de chunks de contexto a usar
        modelo: Modelo da OpenAI a usar

    Returns:
        Dicionário com resposta, contexto usado e fontes
    """
    # 1. RETRIEVAL - Busca contexto relevante
    contextos = buscar_contexto(pergunta, top_k=top_k)

    # 2. AUGMENTED - Monta o prompt com contexto
    contexto_texto = "\n\n".join(
        [f"[Fonte: {c['fonte']}]\n{c['chunk']}" for c in contextos]
    )

    prompt_sistema = """Você é um assistente especializado em responder perguntas com base em informações fornecidas.

REGRAS:
1. Responda APENAS com base no contexto fornecido
2. Se a informação não estiver no contexto, diga "Não encontrei essa informação na base de conhecimento"
3. Cite as fontes quando possível
4. Seja conciso e direto
5. Responda em português"""

    prompt_usuario = f"""CONTEXTO:
{contexto_texto}

PERGUNTA: {pergunta}

RESPOSTA:"""

    # 3. GENERATION - Gera resposta com LLM
    resposta = client.chat.completions.create(
        model=modelo,
        messages=[
            {"role": "system", "content": prompt_sistema},
            {"role": "user", "content": prompt_usuario},
        ],
        temperature=0.3,  # Baixa temperatura para respostas mais factuais
        max_tokens=500,
    )

    return {
        "pergunta": pergunta,
        "resposta": resposta.choices[0].message.content,
        "contextos": contextos,
        "fontes": list(set([c["fonte"] for c in contextos])),
    }


print("✅ Sistema RAG pronto!")

✅ Sistema RAG pronto!


In [12]:
def perguntar(pergunta: str, mostrar_contexto: bool = False, top_k: int = 3) -> dict:
    """
    Interface amigável para fazer perguntas ao sistema RAG.
    """
    print(f"\n{'═' * 70}")
    print(f"❓ PERGUNTA: {pergunta}")
    print(f"{'═' * 70}")

    resultado = gerar_resposta_rag(pergunta, top_k=top_k)

    print(f"\n💬 RESPOSTA:")
    print(f"{resultado['resposta']}")

    print(f"\n📚 FONTES: {', '.join(resultado['fontes'])}")

    if mostrar_contexto:
        print(f"\n{'─' * 70}")
        print("📄 CONTEXTO UTILIZADO:")
        for i, ctx in enumerate(resultado["contextos"], 1):
            print(f"\n[{i}] {ctx['fonte']} (similaridade: {ctx['similaridade']:.1%})")
            print(f"    {ctx['chunk'][:150]}...")

    return resultado


print("✅ Interface de perguntas pronta!")

✅ Interface de perguntas pronta!


## 4. Testando o Sistema RAG

In [7]:
# Teste 1: Pergunta sobre física
perguntar(
    "Quem foi campeão mundial após disputa de pênaltis na final", mostrar_contexto=True
)


══════════════════════════════════════════════════════════════════════
❓ PERGUNTA: Quem foi campeão mundial após disputa de pênaltis na final
══════════════════════════════════════════════════════════════════════

💬 RESPOSTA:
A Argentina foi campeã mundial após a disputa de pênaltis na final da Copa do Mundo FIFA de 2022.

📚 FONTES: Zinedine Zidane, Lionel Messi

──────────────────────────────────────────────────────────────────────
📄 CONTEXTO UTILIZADO:

[1] Lionel Messi (similaridade: 53.4%)
    [Lionel Messi] A final da Copa do Mundo FIFA de 2022, disputada em 18 de dezembro, no Estádio Lusail, no Catar, entrou para a história como uma das de...

[2] Lionel Messi (similaridade: 52.1%)
    [Lionel Messi] Apenas um minuto depois, Mbappé voltou a balançar as redes com um chute de primeira na entrada da área, empatando o confronto em 2–2 e ...

[3] Zinedine Zidane (similaridade: 50.2%)
    [Zinedine Zidane] A partida seguiu empatada, sem muitos chances de perigo, até o fim do tempo nor

{'pergunta': 'Quem foi campeão mundial após disputa de pênaltis na final',
 'resposta': 'A Argentina foi campeã mundial após a disputa de pênaltis na final da Copa do Mundo FIFA de 2022.',
 'contextos': [{'chunk': '[Lionel Messi] A final da Copa do Mundo FIFA de 2022, disputada em 18 de dezembro, no Estádio Lusail, no Catar, entrou para a história como uma das decisões mais marcantes do futebol mundial. Diante de mais de 88 mil espectadores e de uma audiência global estimada em 1,42 bilhão de telespectadores, Argentina e França protagonizaram um confronto caracterizado por sucessivas reviravoltas, atuações individuais de destaque e um desfecho amplamente descrito como dramático. A equipe comandada por Lionel Scaloni dominou o primeiro tempo. Aos 23 minutos, Lionel Messi abriu o placar ao converter um pênalti, após falta cometida sobre Ángel Di María. Aos 36 minutos Di María concluiu um rápido contra-ataque iniciado por Messi, fazendo 2–0 ainda antes do intervalo. No entanto, no segundo

In [ ]:
# Teste 2: Pergunta sobre futebol
perguntar("Quantas Bolas de Ouro o Messi ganhou?")

In [8]:
# Teste 3: Pergunta sobre biologia
perguntar(
    "Qual cientista teve problemas de saúde relacionados ao coração?",
    mostrar_contexto=True,
)


══════════════════════════════════════════════════════════════════════
❓ PERGUNTA: Qual cientista teve problemas de saúde relacionados ao coração?
══════════════════════════════════════════════════════════════════════

💬 RESPOSTA:
Albert Einstein teve problemas de saúde relacionados ao coração, pois entrou em colapso com uma condição cardíaca grave durante uma viagem a Davos, Suíça, em março de 1928. [Fonte: Albert Einstein]

📚 FONTES: Charles Darwin, Albert Einstein

──────────────────────────────────────────────────────────────────────
📄 CONTEXTO UTILIZADO:

[1] Albert Einstein (similaridade: 51.6%)
    [Albert Einstein] No entanto, recusou e escreveu em sua resposta que estava "profundamente comovido" e "ao mesmo tempo triste e envergonhado", pois nã...

[2] Albert Einstein (similaridade: 50.6%)
    [Albert Einstein] Em março de 1928, durante uma viagem a Davos, Suíça, entrou em colapso com uma condição cardíaca grave. Confinado à cama por quatro ...

[3] Charles Darwin (similarida

{'pergunta': 'Qual cientista teve problemas de saúde relacionados ao coração?',
 'resposta': 'Albert Einstein teve problemas de saúde relacionados ao coração, pois entrou em colapso com uma condição cardíaca grave durante uma viagem a Davos, Suíça, em março de 1928. [Fonte: Albert Einstein]',
 'contextos': [{'chunk': '[Albert Einstein] No entanto, recusou e escreveu em sua resposta que estava "profundamente comovido" e "ao mesmo tempo triste e envergonhado", pois não poderia aceitá-la: "Toda a minha vida eu tenho lidado com questões objetivas, daí me falta tanto a aptidão natural e a experiência para lidar corretamente com as pessoas e para o exercício da função oficial. Eu estou muito triste com essas circunstâncias, porque a minha relação com o povo judeu se tornou o meu laço humano mais forte, uma vez que eu consegui compreender a clareza sobre a nossa posição precária entre as nações do mundo". No verão de 1950, seus médicos descobriram que um aneurisma — um vaso sanguíneo fraco — 

In [13]:
# Teste 4: Pergunta que requer síntese de múltiplas fontes
perguntar("Quais cientistas tiveram problemas de saúde significativos?", top_k=5)


══════════════════════════════════════════════════════════════════════
❓ PERGUNTA: Quais cientistas tiveram problemas de saúde significativos?
══════════════════════════════════════════════════════════════════════

💬 RESPOSTA:
Os cientistas que tiveram problemas de saúde significativos foram:

1. **Charles Darwin** - Apresentou problemas incapacitantes no estômago, incluindo vômitos, palpitações e outros sintomas, que se intensificavam durante períodos de estresse.

2. **Isaac Newton** - Sofreu um colapso nervoso em 1693, que incluiu comportamentos estranhos, como o envio de cartas acusatórias a amigos.

3. **Albert Einstein** - Embora não sejam descritos problemas de saúde significativos em seus últimos dias, ele passou por uma autópsia após sua morte, onde seu cérebro foi removido para preservação. 

(Fonte: Charles Darwin, Isaac Newton, Albert Einstein)

📚 FONTES: Albert Einstein, Charles Darwin, Isaac Newton


{'pergunta': 'Quais cientistas tiveram problemas de saúde significativos?',
 'resposta': 'Os cientistas que tiveram problemas de saúde significativos foram:\n\n1. **Charles Darwin** - Apresentou problemas incapacitantes no estômago, incluindo vômitos, palpitações e outros sintomas, que se intensificavam durante períodos de estresse.\n\n2. **Isaac Newton** - Sofreu um colapso nervoso em 1693, que incluiu comportamentos estranhos, como o envio de cartas acusatórias a amigos.\n\n3. **Albert Einstein** - Embora não sejam descritos problemas de saúde significativos em seus últimos dias, ele passou por uma autópsia após sua morte, onde seu cérebro foi removido para preservação. \n\n(Fonte: Charles Darwin, Isaac Newton, Albert Einstein)',
 'contextos': [{'chunk': '[Charles Darwin] No resto de sua vida, ele apresentou problemas incapacitantes no estômago, desencadeando vômitos, palpitações, furúnculos, tremedeiras e outros sintomas. Os episódios se intensificavam durante períodos de stress, co

## 5. RAG com Histórico de Conversa

Vamos criar uma versão mais avançada que mantém o histórico da conversa.

In [ ]:
class ChatRAG:
    """
    Chatbot RAG com memória de conversa.
    """

    def __init__(self, modelo: str = "gpt-4o-mini", top_k: int = 3):
        self.modelo = modelo
        self.top_k = top_k
        self.historico = []
        self.prompt_sistema = """Você é um assistente especializado em responder perguntas com base em informações fornecidas.

REGRAS:
1. Responda APENAS com base no contexto fornecido
2. Se a informação não estiver no contexto, diga "Não encontrei essa informação na base de conhecimento"
3. Cite as fontes quando relevante
4. Seja conciso e direto
5. Responda em português
6. Considere o histórico da conversa para entender referências como "ele", "ela", "isso""""

    def chat(self, pergunta: str) -> str:
        """
        Envia uma mensagem e recebe a resposta.
        """
        # Busca contexto
        contextos = buscar_contexto(pergunta, top_k=self.top_k)

        contexto_texto = "\n\n".join([
            f"[Fonte: {c['fonte']}]\n{c['chunk']}"
            for c in contextos
        ])

        # Monta mensagem com contexto
        mensagem_usuario = f"""CONTEXTO RELEVANTE:
{contexto_texto}

PERGUNTA DO USUÁRIO: {pergunta}"""

        # Adiciona ao histórico
        self.historico.append({"role": "user", "content": mensagem_usuario})

        # Gera resposta
        mensagens = [{"role": "system", "content": self.prompt_sistema}] + self.historico[-10:]  # Últimas 10 mensagens

        resposta = client.chat.completions.create(
            model=self.modelo,
            messages=mensagens,
            temperature=0.3,
            max_tokens=500
        )

        resposta_texto = resposta.choices[0].message.content

        # Adiciona resposta ao histórico
        self.historico.append({"role": "assistant", "content": resposta_texto})

        return resposta_texto

    def limpar_historico(self):
        """Limpa o histórico de conversa."""
        self.historico = []
        print("🗑️ Histórico limpo!")

    def mostrar_historico(self):
        """Mostra o histórico de conversa."""
        print("\n📜 HISTÓRICO DA CONVERSA:")
        print("═" * 50)
        for msg in self.historico:
            role = "👤 Você" if msg["role"] == "user" else "🤖 Assistente"
            # Para mensagens do usuário, mostra apenas a pergunta
            if msg["role"] == "user":
                pergunta = msg["content"].split("PERGUNTA DO USUÁRIO:")[-1].strip()
                print(f"\n{role}: {pergunta}")
            else:
                print(f"\n{role}: {msg['content'][:200]}..." if len(msg['content']) > 200 else f"\n{role}: {msg['content']}")


# Instanciando o chatbot
chat = ChatRAG()
print("✅ ChatRAG inicializado!")

In [ ]:
# Conversa com contexto
print("💬", chat.chat("Quem foi Albert Einstein?"))

In [ ]:
# Pergunta de follow-up usando "ele"
print("💬", chat.chat("Onde ele nasceu?"))

In [ ]:
# Outra pergunta de follow-up
print("💬", chat.chat("E qual foi sua principal contribuição para a física?"))

In [ ]:
# Ver histórico
chat.mostrar_historico()

## 6. Comparação: Com RAG vs Sem RAG

Vamos ver a diferença entre perguntar ao LLM diretamente vs usar RAG.

In [ ]:
def comparar_rag_vs_direto(pergunta: str):
    """
    Compara a resposta do RAG com a resposta direta do LLM.
    """
    print(f"\n{'═' * 70}")
    print(f"❓ PERGUNTA: {pergunta}")
    print(f"{'═' * 70}")

    # Resposta DIRETA (sem RAG)
    resposta_direta = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": pergunta}],
        temperature=0.3,
        max_tokens=300,
    )

    # Resposta com RAG
    resultado_rag = gerar_resposta_rag(pergunta)

    print(f"\n🔴 SEM RAG (conhecimento geral do modelo):")
    print(f"{resposta_direta.choices[0].message.content}")

    print(f"\n🟢 COM RAG (baseado na nossa base de conhecimento):")
    print(f"{resultado_rag['resposta']}")
    print(f"\n   📚 Fontes: {', '.join(resultado_rag['fontes'])}")

In [ ]:
# Teste: Pergunta específica sobre nossa base
comparar_rag_vs_direto("Em que cidade Messi iniciou sua carreira profissional?")

In [ ]:
# Teste: Pergunta sobre detalhes específicos
comparar_rag_vs_direto("Qual foi a causa da morte de Einstein?")

## 7. Experimente você mesmo!

Faça suas próprias perguntas usando o sistema RAG.

In [ ]:
# Limpe o histórico para uma nova conversa
chat.limpar_historico()

In [ ]:
# Digite suas perguntas aqui!
print("💬", chat.chat("Sua pergunta aqui"))

## 💡 O que aprendemos?

1. **RAG completo** - Combinação de busca semântica + LLM para respostas contextualizadas
2. **Prompt engineering** - Como estruturar prompts para respostas mais precisas
3. **Histórico de conversa** - Manter contexto entre perguntas
4. **RAG vs Direto** - Vantagens de usar RAG para perguntas específicas

### 🚀 Próximos passos para um sistema de produção:

1. **Vector Database** - Usar Pinecone, Weaviate, Chroma ou FAISS para escalar
2. **Reranking** - Usar modelos de reranking para melhorar a precisão
3. **Chunking avançado** - Estratégias como recursive splitting, semantic chunking
4. **Avaliação** - Métricas como RAGAS para avaliar qualidade das respostas
5. **Streaming** - Respostas em tempo real para melhor UX
6. **Hybrid Search** - Combinar busca semântica com busca por keywords (BM25)